In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import exp
from pyspark.sql.functions import max,sum,count
from pyspark.ml.classification import LogisticRegression  
import matplotlib.pyplot as plt
import numpy as np
spark = SparkSession.builder.appName('Wine Quality Analysis').getOrCreate()

In [2]:
# Read the original data
df1 = spark.read.csv("winequality-white.csv",inferSchema=True,header=True)
df2 = spark.read.csv("winequality-red.csv",inferSchema=True,header=True)

In [5]:
# Import the relevant packages.
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

assembler = VectorAssembler(inputCols=['residual sugar','chlorides', 'free sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'], outputCol="features")

In [6]:
from pyspark.ml import Pipeline

# Then go through our steps. It's essentially sequential to the above.
pipeline = Pipeline(stages=[assembler])

# Now that we've got a number of steps, let's apply it to the DataFrame.
pipeline_model = pipeline.fit(df1)

# Incorporate results into a new DataFrame.
pipe_df = pipeline_model.transform(df1)

# Remove all variables other than features and label. 
pipe_df = pipe_df.select('quality', 'features')

from pyspark.ml.classification import LogisticRegression

# Split our data. Note that the new DataFrame is being used.
train_data, test_data = pipe_df.randomSplit([0.7,0.3])
print("Training Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 3467
Test Dataset Count: 1431


In [10]:
## NN
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
#from sklearn.neural_network import MLPClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [11, 9, 8, 7 , 6]

# create the trainer and set its parameters
nn = MultilayerPerceptronClassifier(labelCol="quality", featuresCol="features",maxIter=100, layers=layers, blockSize=128, seed=1234)
#nn = MultilayerPerceptronClassifier(labelCol="quality", featuresCol="features",layers=layers)

# Train a NN model.
#nn = MLPClassifier(labelCol="quality", featuresCol="features")

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[ nn])

In [11]:
# Train model.  This also runs the indexers.
nn_model = pipeline.fit(train_data)

# Make predictions.
nn_result = nn_model.transform(test_data)

# Select example rows to display.
nn_result.select("prediction", "quality", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="quality", predictionCol="prediction", metricName="accuracy")
nn_accuracy = evaluator.evaluate(nn_result) 
print('NNClassifier accuracy: ' + str(nn_accuracy))
nn_result.groupBy(nn_result['quality'].alias('OriginYes')).agg(count("quality").alias("Total_Count"),sum("prediction").alias("Prediction_Yes_Count")).show()

Py4JJavaError: An error occurred while calling o235.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 12, localhost, executor driver): java.lang.ArrayIndexOutOfBoundsException
	at java.lang.System.arraycopy(Native Method)
	at org.apache.spark.ml.ann.DataStacker$$anonfun$5$$anonfun$apply$3$$anonfun$apply$4.apply(Layer.scala:628)
	at org.apache.spark.ml.ann.DataStacker$$anonfun$5$$anonfun$apply$3$$anonfun$apply$4.apply(Layer.scala:627)
	at scala.collection.immutable.List.foreach(List.scala:381)
	at org.apache.spark.ml.ann.DataStacker$$anonfun$5$$anonfun$apply$3.apply(Layer.scala:627)
	at org.apache.spark.ml.ann.DataStacker$$anonfun$5$$anonfun$apply$3.apply(Layer.scala:623)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemorySto